This is the Python implementation of the 3D-RFT Code based on the framework
proposed by Agarwal et al. --> https://doi.org/10.1073/pnas.2214017120

In [4]:
######## IMPORTS ########
import math

import numpy as np
import scipy as sp

from src import rft_functions


ModuleNotFoundError: No module named 'src'